# Python wrangling

### We'll be going through some examples and uses of basic Python tools for handling data

In [1]:
import pandas as pd 
import numpy as np
% matplotlib inline

np.set_printoptions(suppress=True) ##Suppress scientific notation


UsageError: Line magic function `%` not found.


The dataset we'll be using might be familiar for some of you- we'll be taking a look at the flight delay data used at the IML hackathon.

First, we need to read the data into memory.
Other than reading CSV files, Pandas can read other file formats, data through API calls, use SQL to connect and read from database tables, parse tables from HTML and even read data from your clipboard!

In [ ]:
df = pd.read_csv("train_data.csv")

Before we do anything fancy, let's start by taking a look at our data:

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Note that we have several ways to simply take a look at our dataset before handling it in a more thorough way. When we'll delve more deeply into the EDA process we'll look at some more ways to do so.

As you might know, Pandas is built on top of Numpy arrays. In case we want to start messing with the underlying array, we can call: 

In [ ]:
df.to_numpy()

Notice that, when .info() was called, we saw each column's datatype. Take note of it- handling large datasets can take a toll on your memory usage and performance, and making sure the correct Dtype for each column is being used is a good practice.

#### Handling our dataset

Great, now we have some data in-memory. What do we do with it? 

Accessing our dataframe can be done in two main way. Using location indices with the .loc and .iloc properties:

In [ ]:
df.loc[:2,['DayOfWeek','Origin']]

In [ ]:
df.iloc[:2,2]

And accessing columns on a named basis:

In [ ]:
df["Origin"]

In [ ]:
df[["Origin","DayOfWeek"]]

#### Aggregate functions and filtering, take 1

Cool, so we can select columns. But what now?

Say we wanted to take a closer look at the different origins. Specifically,
we wanted to see which airports are 'busy'. How should we define this question more succintly so that we can try and answer this through our data?

Aggregate functions are functions which are applied to different parts of a given dataset. Let's take a look so it becomes clearer. Say we wanted to check how many flights took off from JFK- as it's a major NYC airport, we assume it's probably a good example of a busy airport. 

One thing we could do is just filter our dataset to flights originating from JFK, and look at the resulting dataframe's shape:

In [ ]:
df[df["Origin"]=="JFK"].shape

So, 10394 records are of flight from JFK, with our 17 columns of features.

Take a moment to ponder if this is a scaleable way to do things. If later down the line we wanted to use this as a feature in our project, should we do this for every airport?

Considering i wrote "take 1" in our title, you can safely assume "no".


#### Aggregate functions, take 2

So, say we wanted to get a count for every airport around. 
Applying an aggragate function is essentialy grouping our data by specific properties and applying some sort of function- like this:

In [ ]:
df.groupby("Origin").count()

Notice that we got 16 equal columns- the .count methods did not distinguish between the existing columns. It also turned our index to the Origin columns.
In order to return to a previous,more easy-to-use format, we can do the following:

In [ ]:
df.groupby("Origin").count()['DayOfWeek'].reset_index()

Selecting a single column, denoted here as "DayOfWeek", simply lets us choose one of the 16 identical columns- any existing colum will do. 


At this point we should note that the amount columns grouped by is arbitrary- we could group by any set of columns (as long as they exist). We just have to make sure that whatever we're doing makes sense:


In [ ]:
df.groupby(["Origin","Dest"]).count()['DayOfWeek'].reset_index()

We still have a lot of rows to look at. At this point, we could consider visualizing our data to gain better insights. Let's try this now.

#### Visualizing data

Much like most over tasks, Python has a plethora of data viz libraries:
- Matplotlib (which we've seen at previous courses)
- Seaborn, which is written on top of matplotlib and can usually get a nicer looking baseline with less of a hassle
- Plotly, which allows interactive visualization

and many, many more

Let's take a look- we'll import seaborn and try and make some visualization to better understand what's going on with our data.

In [ ]:
import seaborn as sns

In [ ]:
to_plot= df.groupby("Origin").count()['DayOfWeek'].reset_index()

In [ ]:
sns.boxplot(y="DayOfWeek",data=to_plot,)

Well, that wasn't really to informative- our data is highly concentrated and it's a bit hard to comprehend what's going on. 
Let's use this opportunity to see how we apply functions and transformations to our data in a more basic, non-aggregate way.
One possible way to improve our visualization is to apply a log transformation to our count column. Note that this lowers the explainability of our plot a bit, but it's fine (for now).

We could, for example, create a new column to preserve our current existing data. Note that as pandas DataFrames and Series are based on numpy objects, we could make use of its implementations:

In [ ]:
to_plot['log_days']=np.log( to_plot.DayOfWeek)

In [ ]:
sns.boxplot(y="log_days",data=to_plot,)

Another option is applying a method axis- wise. For example, lets try normalizing our dataset to have a mean of 0 and sd of 1:

In [ ]:
to_plot['normalized_count']=to_plot['DayOfWeek']  -to_plot['DayOfWeek'].mean()  
to_plot['normalized_count'] =to_plot['normalized_count'] / to_plot['DayOfWeek'].std()

In [ ]:
to_plot['normalized_count'].mean()

In [ ]:
to_plot['normalized_count'].std()

In [ ]:
sns.boxplot(y="normalized_count",data=to_plot,)

####  Merging data

Finally, let's talk about using more than a single dataset.
For example, let's load up our provided weather data:

In [ ]:
weather_df = pd.read_csv("all_weather_data.csv")

In [ ]:
weather_df.head()

As you might remember, this dataset contains weather conditions for different stations on different days.
What we're going to do is join together our two dataframes to one larger and more informative one.
Joins, which some of you might recognize from SQL, define the process of merging (or... joining) two datasets together.
The two main types of joins are Inner and Outer joins, with outer joins have several subtypes.

<img src="https://www.dofactory.com/Images/sql/sql-joins.png" width=500 height=500 />
<center>How did we get this far without Venn diagrams?


So, how is a join performed and what's the difference?

When declaring a join, we use what we'd call keys. We're going to look for corresponding columns in both joined tables, which have corresponding values, and 'join' using them. Don't worry, we'll take a look at it in a minute.
The types of joins are as follows:

- Inner join: Only keep rows with matches in both tables. This means that, for example, if we had an airport without any weather data or weather data for a non tracked airport (or worse, for a wrongly written airport name) - we'll lose it when joining. This is not necessarily a bad thing- but something to note.
- Outer join: Allow missing values on at least one side. A left or right join allows no-matches on the relvant side, and a full join allows them on both tables.


Pandas does joining using the .merge method. We'll perform our example join using the origin station and day. First, we'll convert our Date columns to datetime, and perform our join:

In [ ]:
df.FlightDate=pd.to_datetime(df.FlightDate)

In [ ]:
weather_df.day=pd.to_datetime(weather_df.day)

In [ ]:
df.merge(weather_df,left_on=['Origin',"FlightDate"],right_on = ['station','day'])

Yielding a much richer dataset for us to work with!